In [1]:
import pandas as pd

In [2]:
import loader as load
ge_phylum_stage = load.loadGEOverlappingTCMA("phylum", includeStage=True)


In [3]:
import processor as pr
stad_ge_phylum_stage = ge_phylum_stage[ge_phylum_stage["project"]=="STAD"]
# print(stad_ge_phylum_stage["stage"].values)
stage_dictionary = {
    "I":0,
    "IA":0,
    "IB":0,
    "II":1,
    "IIA":1,
    "IIB":1,
    "III":2,
    "IIIA":2,
    "IIIB":2,
    "IIIC":2,
    "IV":3,
}

def convertStage(stage):
    st = str(stage)[6:]
    return stage_dictionary[st]
stad_ge_phylum_stage["stage"] = stad_ge_phylum_stage.apply(lambda row: convertStage(row.stage) if row.stage==row.stage else row.stage, axis=1)   
print(stad_ge_phylum_stage["stage"].values)
# load.attachTumorStatus(stad_ge_phylum_stage)
x, y = pr.splitData(stad_ge_phylum_stage, "stage")

[ 0.  0.  0.  2.  3.  2.  3.  1.  1.  1.  1.  3.  1.  1.  3.  2.  2. nan
  2.  2.  0. nan  1. nan nan  0.  2.  2. nan  0.  3.  2. nan  3.  2.  2.
  2.  1.  0.  3.  2.  3.  1.  3. nan  1. nan  2. nan  1.  1.  3.  2.  2.
  2.  0.  0.  2.  2.  3.  1.  0.  2.  1.  0.  2.  1. nan nan  1.  0.  2.
  1.  3.  0.  1.  0.  2.  2.  3.  1.  1. nan  1. nan  2.  2.  0.  1.  3.
  0.  1.  0.  3.  2.  1. nan  3.  1.  1.  0.  1.  1.  2.  1.  3.  3.  1.
  0.  0.  1.  0.  1.  3.  1. nan  1.  1.  1.  1.  2.  1.]


C:\Users\Ameno\AppData\Local\Temp\ipykernel_10964\3993372423.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stad_ge_phylum_stage["stage"] = stad_ge_phylum_stage.apply(lambda row: convertStage(row.stage) if row.stage==row.stage else row.stage, axis=1)


In [6]:
import predictor as pred

tcma_genus = load.loadTCMA("genus")
tcma_genus_aak_ge = load.loadGEOverlappingTCMA("genus")
aak_ge = load.loadGEWithClinical()
data = [tcma_genus, aak_ge, tcma_genus_aak_ge]
files = ["tcma_gen", "aak_ge", "tcma_gen_aak_ge"]

c:\Users\Ameno\Desktop\Life\Thesis\Host-Microbial-Omics-Integration\loader.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aak_ge_clinical_types["portion_id"] = aak_ge_clinical_types.apply(lambda row: str(row.portion_id)[:-4], axis=1)
c:\Users\Ameno\Desktop\Life\Thesis\Host-Microbial-Omics-Integration\loader.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aak_ge_clinical_types.drop_duplicates(["portion_id"], inplace=True)


In [11]:
for i, d in enumerate(data):
    load.attachTumorStatus(d)
    final_report = ""
    for c in ["COAD", "ESCA", "HNSC", "READ", "STAD"]:   
        if i == 2 and  c == "READ":
            continue
        x, y = pr.splitData(d, target="tumor", project=c)
        cur_report = pred.runCrossValidation(x, y)
        final_report += f"Summary data for: {c}\n"
        description = ",".join([k + ": " + str(cur_report[k]) for k in cur_report])
        final_report += description + "\n"
        load.saveDescriptor(final_report, fr'Data\Descriptor\{files[i]}_tmr_pred.txt')
# print("Final final_report", final_report)


c:\Users\Ameno\Anaconda3\envs\bacteria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ameno\Anaconda3\envs\bacteria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ameno\Anaconda3\envs\bacteria\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Amen